In [1]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score

df_features = pd.read_json('/kaggle/input/indoml-phase2/train.features',lines=True)
df_labels = pd.read_json('/kaggle/input/indoml-phase2/train.labels',lines=True)

In [2]:
df = pd.merge(df_features,df_labels,on="indoml_id")

In [3]:
# df = df[:100]

In [4]:
from sklearn.preprocessing import LabelEncoder
group_encoder = LabelEncoder()
supergroup_encoder = LabelEncoder()
module_encoder = LabelEncoder()
brand_encoder = LabelEncoder()

# Fit and transform each column
df['group'] = group_encoder.fit_transform(df['group'])
df['supergroup'] = supergroup_encoder.fit_transform(df['supergroup'])
df['module'] = module_encoder.fit_transform(df['module'])
df['brand'] = brand_encoder.fit_transform(df['brand'])

In [21]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
# from transformers import DebertaModel, DebertaTokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from torch.distributions import Categorical
import numpy as np
import time
from torch.profiler import profile, record_function, ProfilerActivity

In [6]:
# For using the new one
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# For loading the saved one
# model_dir = "/kaggle/input/new-transformer-experiment-12-embedding-tmp/New_model"
# tokenizer = DebertaTokenizer.from_pretrained(model_dir)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
MAX_LENGTH = 12
BATCH_SIZE = 64
LEARNING_RATE = 5e-5
NUM_EPOCHS = 10
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device=DEVICE
PATIENCE = 5  # Early stopping patience
PATIENCE_LR = 3  # Reduce LR on plateau patience

In [8]:
from sklearn.metrics import accuracy_score
import torch

In [9]:
class ProductDataset(Dataset):
    def __init__(self, texts, labels1, labels2, labels3, labels4):
        self.texts = texts
        self.labels1 = labels1
        self.labels2 = labels2
        self.labels3 = labels3
        self.labels4 = labels4
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors="pt")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        item = {key: val[idx].to(DEVICE) for key, val in self.encodings.items()}
        item['labels1'] = torch.tensor(self.labels1[idx], device=DEVICE)
        item['labels2'] = torch.tensor(self.labels2[idx], device=DEVICE)
        item['labels3'] = torch.tensor(self.labels3[idx], device=DEVICE)
        item['labels4'] = torch.tensor(self.labels4[idx], device=DEVICE)
        return item
        
def compute_accuracy(preds, labels):
    # Convert each tensor in the list to numpy arrays
    preds_np = [p.cpu().numpy() for p in preds]
    labels_np = [l.cpu().numpy() for l in labels]
    # Individual accuracies for each of the 4 labels
    accuracies = [accuracy_score(labels_np[i], preds_np[i]) for i in range(4)]
    # Overall accuracy where all 4 labels match
    overall_accuracy = accuracy_score(
        np.all([labels_np[i] == preds_np[i] for i in range(4)], axis=0), 
        np.ones(len(labels_np[0]))
    )
    # Return the 5 accuracies (4 individual, 1 overall)
    return accuracies + [overall_accuracy]

In [10]:
# Split data
train_texts, val_texts, train_labels1, val_labels1, train_labels2, val_labels2, train_labels3, val_labels3, train_labels4, val_labels4 = train_test_split(
    df['description'], 
    df['supergroup'], 
    df['group'], 
    df['module'], 
    df['brand'], 
    test_size=0.2, 
    random_state=42
)

train_dataset = ProductDataset(
    train_texts.tolist(), 
    train_labels1.tolist(), 
    train_labels2.tolist(), 
    train_labels3.tolist(), 
    train_labels4.tolist()
)

val_dataset = ProductDataset(
    val_texts.tolist(), 
    val_labels1.tolist(), 
    val_labels2.tolist(), 
    val_labels3.tolist(), 
    val_labels4.tolist()
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from transformers import DebertaModel, DebertaTokenizer
from transformers import AutoModel, AutoTokenizer
from torch.distributions import Categorical
import os

class AdvancedHierarchicalClassifier(nn.Module):
    def __init__(self, num_supergroups, num_groups, num_modules, num_brands):
        super().__init__()
        # Load the base model
        self.deberta = AutoModel.from_pretrained(
            "microsoft/deberta-v3-base",
            num_labels=1,  # Set to None to get the base model without classification head
            output_hidden_states=True  # Enable output of hidden states
        )
        
        # Remove the classification head if it exists
        if hasattr(self.deberta, 'classifier'):
            delattr(self.deberta, 'classifier')
        
        hidden_size = self.deberta.config.hidden_size
        
        # Classification heads for different levels
        self.supergroup_classifier = nn.Linear(hidden_size, num_supergroups)
        self.group_classifier = nn.Linear(hidden_size, num_groups)
        self.module_classifier = nn.Linear(hidden_size, num_modules)
        self.brand_classifier = nn.Linear(hidden_size, num_brands)

        # Pooler layer
        self.pooler = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh()
        )

    def forward(self, input_ids, attention_mask):
        # Get the base model outputs
        outputs = self.deberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        
        # Get the last hidden state and apply pooling
        last_hidden_state = outputs.hidden_states[-1]
        cls_token = last_hidden_state[:, 0, :]  # Get [CLS] token
        pooled_output = self.pooler(cls_token)
        
        # Generate predictions for each level
        supergroup_logits = self.supergroup_classifier(pooled_output)
        group_logits = self.group_classifier(pooled_output)
        module_logits = self.module_classifier(pooled_output)
        brand_logits = self.brand_classifier(pooled_output)
        
        return {
            'supergroup': supergroup_logits,
            'group': group_logits,
            'module': module_logits,
            'brand': brand_logits
        }

    def get_embeddings(self, input_ids, attention_mask):
        outputs = self.deberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        last_hidden_state = outputs.hidden_states[-1]
        cls_token = last_hidden_state[:, 0, :]
        return self.pooler(cls_token)

In [23]:
class JointAccuracyTrainer:
    def __init__(self, model, supervised_optimizer, device):
        self.model = model
        self.supervised_optimizer = supervised_optimizer
        self.device = device
        self.best_accuracy = 0
        self.best_model_state = None

    def supervised_step(self, batch, true_labels):
        self.supervised_optimizer.zero_grad()
        
        # Forward pass through the model
        outputs = self.model(
            batch['input_ids'],
            batch['attention_mask']
        )
        
        # Compute loss for each level
        total_loss = self.compute_joint_loss(outputs, true_labels)
        
        # Backward pass
        total_loss.backward()
        self.supervised_optimizer.step()
        
        return total_loss.item()

    def compute_joint_loss(self, outputs, true_labels):
        losses = []
        weights = {
            'supergroup': 1.0,
            'group': 1.0,
            'module': 1.0,
            'brand': 1.0
        }
        
        for level, logits in outputs.items():
            if level in true_labels:
                loss = F.cross_entropy(logits, true_labels[level])
                losses.append(weights[level] * loss)
        
        return sum(losses)

    def compute_accuracy(self, outputs, true_labels):
        accuracies = {}
        all_correct = True
        for level, logits in outputs.items():
            if level in true_labels:
                predictions = torch.argmax(logits, dim=1)
                correct = (predictions == true_labels[level]).float().mean()
                accuracies[level] = correct.item()
                all_correct &= (predictions == true_labels[level]).all().item()
        accuracies['item'] = float(all_correct)
        return accuracies

    def save_best_model(self, current_accuracy):
        if current_accuracy > self.best_accuracy:
            self.best_accuracy = current_accuracy
            self.best_model_state = {
                key: value.cpu().clone() for key, value in self.model.state_dict().items()
            }

In [24]:
# Usage
model = AdvancedHierarchicalClassifier(num_supergroups=32, num_groups=228, num_modules=449, num_brands=5679).to(DEVICE)
# For loading the model saved after pre-training the saved dict
model_path = '/kaggle/input/debertav3-12-embedding-tmp/New_model/model.pth'
model.load_state_dict(torch.load(model_path))

/tmp/ipykernel_30/1464995192.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [26]:
# Profile on a single sample before starting the full training
model.train()
sample_batch = next(iter(train_loader))  # Get one sample batch
sample_batch = {k: v.to(device) for k, v in sample_batch.items()}
true_labels = {
    'supergroup': sample_batch['labels1'],
    'group1': sample_batch['labels2'],
    'group2': sample_batch['labels3'],
    'group3': sample_batch['labels4']
}
device = next(model.parameters()).device
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
trainer = JointAccuracyTrainer(model, optimizer, device)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], with_flops=True) as prof:
    _ = trainer.supervised_step(sample_batch, true_labels)

print(prof.key_averages().table(sort_by="flops",row_limit=10))
print("GFLOPs during training") #GigaFLOPs
print(sum(k.flops for k in prof.key_averages())/1e9)

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  Total MFLOPs  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               aten::mm         2.89%       5.985ms         2.89%       5.985ms      30.535us       0.000us         0.00%       0.000us       0.000us           196    290067.579  
                                            aten::addmm         2.00%       4.131ms         2.60%       5.378ms      53.248us       0.000us         0.0

In [12]:
# TODO, make a check on item accuracy, if the test accuracy is good one

def train_and_evaluate(model, train_loader, val_loader, num_epochs):
    device = next(model.parameters()).device
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    trainer = JointAccuracyTrainer(model, optimizer, device)
    
    for epoch in range(num_epochs):
        model.train()
        total_sup_loss = 0
        train_accuracies = {
            'supergroup': 0.0,
            'group': 0.0,
            'module': 0.0,
            'brand': 0.0,
            'item': 0.0
        }

        # Training loop
        for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
            batch = {k: v.to(device) for k, v in batch.items()}
            true_labels = {
                'supergroup': batch['labels1'],
                'group': batch['labels2'],
                'module': batch['labels3'],
                'brand': batch['labels4']
            }

            sup_loss = trainer.supervised_step(batch, true_labels)
            total_sup_loss += sup_loss

            # Compute training accuracies
            with torch.no_grad():
                outputs = model(batch['input_ids'], batch['attention_mask'])
                batch_accuracies = trainer.compute_accuracy(outputs, true_labels)
                for level in train_accuracies:
                    train_accuracies[level] += batch_accuracies.get(level, 0.0)

        # Calculate average training accuracies
        for level in train_accuracies:
            train_accuracies[level] /= len(train_loader)

        # Validation loop
        model.eval()
        val_accuracies = {
            'supergroup': 0.0,
            'group': 0.0,
            'module': 0.0,
            'brand': 0.0,
            'item': 0.0
        }
        val_loss = 0

        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Validation'):
                batch = {k: v.to(device) for k, v in batch.items()}
                true_labels = {
                    'supergroup': batch['labels1'],
                    'group': batch['labels2'],
                    'module': batch['labels3'],
                    'brand': batch['labels4']
                }

                outputs = model(batch['input_ids'], batch['attention_mask'])
                batch_accuracies = trainer.compute_accuracy(outputs, true_labels)
                val_loss += trainer.compute_joint_loss(outputs, true_labels).item()

                for level in val_accuracies:
                    val_accuracies[level] += batch_accuracies.get(level, 0.0)

        # Calculate average validation accuracies and loss
        for level in val_accuracies:
            val_accuracies[level] /= len(val_loader)
        val_loss /= len(val_loader)

        # Save best model based on average validation accuracy
        avg_val_accuracy = sum(val_accuracies.values()) / len(val_accuracies)
        trainer.save_best_model(avg_val_accuracy)
        
        # Print epoch results
        print(f'\nEpoch {epoch + 1} Results:')
        print(f'Training Loss: {total_sup_loss / len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss:.4f}')
        print('\nTraining Accuracies:')
        for level, acc in train_accuracies.items():
            print(f'{level}: {acc:.4f}')
        print('\nValidation Accuracies:')
        for level, acc in val_accuracies.items():
            print(f'{level}: {acc:.4f}')
        print('-' * 50)
    
    # Load the best model state at the end of training
    if trainer.best_model_state is not None:
        model.load_state_dict(trainer.best_model_state)

train_and_evaluate(model, train_loader, val_loader, num_epochs=NUM_EPOCHS)

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

/tmp/ipykernel_30/34913142.py:164: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
Epoch 1/10:   3%|▎         | 217/7023 [01:07<3

KeyboardInterrupt: 

In [ ]:
import os
# Define the directory to save the model and tokenizer
save_directory = "New_model"
# Create the directory if it doesn't exist

if not os.path.exists(save_directory):
    os.makedirs(save_directory)
# Save the model's state dictionary

model_save_path = os.path.join(save_directory, "model.pth")
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")
# Save the tokenizer

tokenizer.save_pretrained(save_directory)
print(f"Tokenizer saved to {save_directory}")

In [13]:
df_test_feat = pd.read_json('/kaggle/input/indoml-phase2/final_test_data.features',lines=True)

In [14]:
df_test_feat.head()

,indoml_id,description,retailer,price
0,0,14 in hybrid blade,wilko,4.50
1,1,2 pk vent stick a fres,noshify,0.69
2,2,4 tyrefix 450 ml,noshify,2.99
3,3,4 x 4 tyrefix 450 ml,noshify,2.99
4,4,5 l adbluescr diesel vehicles,noshify,4.99


In [27]:
def predict(model, tokenizer, text):
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        padding='max_length', 
        max_length=MAX_LENGTH
    ).to(DEVICE)
    with torch.no_grad():
        start = time.time()
        with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],with_flops=True) as prof:
            outputs = model(inputs['input_ids'], inputs['attention_mask'])

            # Adjust outputs if it's a tuple or dictionary
            if isinstance(outputs, tuple):
                logits = outputs[0]  # Assuming the first element contains logits
            elif isinstance(outputs, dict):
                logits = list(outputs.values())  # Assuming values in dict are tensors
            else:
                logits = outputs  # If it's already a tensor list

            # Generate predictions
            predictions = [torch.argmax(logit, dim=1).item() for logit in logits]

        print("Inference time :"+str(time.time()-start))
        #print(prof.key_averages().table(sort_by="flops",row_limit=10))
        print("GFLOPs during testing") #GigaFLOPs
        print(sum(k.flops for k in prof.key_averages())/1e9)
    
    return predictions

# Example prediction
sample_text = "14 in hybrid blade"
predictions = predict(model, tokenizer, sample_text)
print(f"Supergroup: {predictions[0]}, Group: {predictions[1]}, Module: {predictions[2]}, Brand: {predictions[3]}")

Inference time :0.5156702995300293
GFLOPs during testing
16.781984352
Supergroup: 16, Group: 114, Module: 212, Brand: 4078


In [16]:
# test_tmp = df_test_feat[:5]

In [18]:
def make_test_pred_and_save(df_test_feat):
    supergroups_list = []
    groups_list = []
    modules_list = []
    brands_list = []
    indoml_id_list = range(len(df_test_feat))
    length_df = df_test_feat.shape[0]
    
    with torch.no_grad():
        for i in range(length_df):
            if i % 1000 == 0:
                print(f"Processing {i} of {length_df - 1}")

            # Tokenize input
            inputs = tokenizer(
                df_test_feat.iloc[i].description, 
                return_tensors="pt", 
                truncation=True, 
                padding='max_length', 
                max_length=MAX_LENGTH
            ).to(DEVICE)

            # Get model outputs
            outputs = model(inputs['input_ids'], inputs['attention_mask'])

            # Adjust outputs if it's a tuple or dictionary
            if isinstance(outputs, tuple):
                logits = outputs[0]  # Assuming the first element contains logits
            elif isinstance(outputs, dict):
                logits = list(outputs.values())  # Assuming values in dict are tensors
            else:
                logits = outputs  # If it's already a tensor list

            # Generate predictions
            predictions = [torch.argmax(logit, dim=1).item() for logit in logits]

            # Append predictions to respective lists
            supergroups_list.append(predictions[0])
            groups_list.append(predictions[1])
            modules_list.append(predictions[2])
            brands_list.append(predictions[3])
            
    # Map predictions back to names using encoders
    try:
        supergroups_names = supergroup_encoder.inverse_transform(supergroups_list)
    except ValueError as e:
        print(f"Error in supergroups: {e}")
        supergroups_names = ['Unknown' if x not in supergroup_encoder.classes_ else x for x in supergroups_list]
    
    try:
        groups_names = group_encoder.inverse_transform(groups_list)
    except ValueError as e:
        print(f"Error in groups: {e}")
        groups_names = ['Unknown' if x not in group_encoder.classes_ else x for x in groups_list]
    
    try:
        modules_names = module_encoder.inverse_transform(modules_list)
    except ValueError as e:
        print(f"Error in modules: {e}")
        modules_names = ['Unknown' if x not in module_encoder.classes_ else x for x in modules_list]
    
    try:
        brands_names = brand_encoder.inverse_transform(brands_list)
    except ValueError as e:
        print(f"Error in brands: {e}")
        brands_names = ['Unknown' if x not in brand_encoder.classes_ else x for x in brands_list]
    
    # Create a DataFrame with predictions
    predictions_df = pd.DataFrame({
        'indoml_id': indoml_id_list,
        'supergroup': supergroups_names,
        'group': groups_names,
        'module': modules_names,
        'brand': brands_names
    })
    
    # Save the predictions as a JSON file
    predictions_df.to_json('/kaggle/working/predictions.predict', orient='records', lines=True)
    print("predictions.predict saved")
        
print(make_test_pred_and_save(df_test_feat))

Processing 0 of 184663
Processing 1000 of 184663
Processing 2000 of 184663
Processing 3000 of 184663
Processing 4000 of 184663
Processing 5000 of 184663
Processing 6000 of 184663
Processing 7000 of 184663
Processing 8000 of 184663
Processing 9000 of 184663
Processing 10000 of 184663
Processing 11000 of 184663
Processing 12000 of 184663
Processing 13000 of 184663
Processing 14000 of 184663
Processing 15000 of 184663
Processing 16000 of 184663
Processing 17000 of 184663
Processing 18000 of 184663
Processing 19000 of 184663
Processing 20000 of 184663
Processing 21000 of 184663
Processing 22000 of 184663
Processing 23000 of 184663
Processing 24000 of 184663
Processing 25000 of 184663
Processing 26000 of 184663
Processing 27000 of 184663
Processing 28000 of 184663
Processing 29000 of 184663
Processing 30000 of 184663
Processing 31000 of 184663
Processing 32000 of 184663
Processing 33000 of 184663
Processing 34000 of 184663
Processing 35000 of 184663
Processing 36000 of 184663
Processing 370